In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [4]:
def coletar_links(url, limite=5000):
    # Fila contém uma série de links (todos os links das páginas acessadas). Set contém os links únicos já visitados.
    fila = [url]
    visitados = set()
    salvos = 0

    # Se menos de 5k links foram guardados, conitnua execução
    while salvos < limite:
        try:
            # Acessa o último link inserido na fila
            atual = fila.pop()

            # Verifica se esse link já foi acessado. Se não, continua a execução e adiciona a lista de visitados.
            if atual in visitados: 
                continue
            visitados.add(atual)

            # Carrega a página atual.
            req = requests.get(atual)
            soup = BeautifulSoup(req.text, 'html.parser')

            titulo = soup.select(".mw-page-title-main")
            nome_arquivo = titulo[0].text

            with open(f"../pag/{nome_arquivo}.html", "w", encoding="utf-8") as file:
                conteudo = req.content.decode("utf-8")
                file.write(conteudo)
            salvos += 1

            # Itera sobre todos os links da página atual.
            for link in soup.find_all('a', href=True):

                # Verifica se a página é um verbete.
                pagina = link['href']
                if pagina.startswith('/wiki/') and ':' not in pagina:

                    # Se algum dos links da página carregada já tiver sido visitada, pula.
                    link_atual = 'https://pt.wikipedia.org' + pagina
                    if link_atual in visitados:
                        continue
                    # Adiciona todos que forem verbetes à fila. O último link coletado será a próxima página a ser aberta.
                    fila.append('https://pt.wikipedia.org' + pagina)

            print(salvos, end="\r")
            time.sleep(0.6)

        # Evita páginas sem título (titulo[0].text)
        except IndexError:
            continue

        # Evita erros de mal formatação de título
        except FileNotFoundError:
            continue

    return list(visitados)

link = coletar_links('https://pt.wikipedia.org/')


KeyboardInterrupt: 